In [ ]:
#pip install pyspark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, count, when
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
spark = SparkSession.builder.appName("Game_Recommendation_System").getOrCreate()

In [ ]:
# Reading Data
recom = spark.read.csv("/content/recommendations.csv", header=True, inferSchema=True)
games = spark.read.csv("/content/games.csv", header=True, inferSchema=True)

In [ ]:
games.show(10)

+------+--------------------+------------+----+-----+-----+---------------+--------------+------------+-----------+--------------+--------+----------+
|app_id|               title|date_release| win|  mac|linux|         rating|positive_ratio|user_reviews|price_final|price_original|discount|steam_deck|
+------+--------------------+------------+----+-----+-----+---------------+--------------+------------+-----------+--------------+--------+----------+
| 13500|Prince of Persia:...|  2008-11-21|true|false|false|  Very Positive|            84|        2199|       9.99|          9.99|     0.0|      true|
| 22364|BRINK: Agents of ...|  2011-08-03|true|false|false|       Positive|            85|          21|       2.99|          2.99|     0.0|      true|
|113020|Monaco: What's Yo...|  2013-04-24|true| true| true|  Very Positive|            92|        3722|      14.99|         14.99|     0.0|      true|
|226560|  Escape Dead Island|  2014-11-18|true|false|false|          Mixed|            61|    

In [ ]:
recom.show(10)

+-------+-------+-----+-------------------+--------------+-----+-------+---------+
| app_id|helpful|funny|               date|is_recommended|hours|user_id|review_id|
+-------+-------+-----+-------------------+--------------+-----+-------+---------+
| 975370|      0|    0|2022-12-12 00:00:00|          true| 36.3|  47199|        0|
| 304390|      4|    0|2017-02-17 00:00:00|         false| 11.5|   2376|        1|
|1085660|      2|    0|2019-11-17 00:00:00|          true|336.5| 230757|        2|
| 703080|      0|    0|2022-09-23 00:00:00|          true| 27.4| 235736|        3|
| 526870|      0|    0|2021-01-10 00:00:00|          true|  7.9|  21721|        4|
| 306130|      0|    0|2021-10-10 00:00:00|          true|  8.6|  41543|        5|
| 238960|      0|    0|2017-11-25 00:00:00|          true|538.8|  80787|        6|
|    730|      0|    0|2021-11-30 00:00:00|         false|157.5|  57879|        7|
| 255710|      0|    0|2021-05-21 00:00:00|          true| 18.7| 321815|        8|
| 28

In [ ]:
games.printSchema()

root
 |-- app_id: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- win: boolean (nullable = true)
 |-- mac: boolean (nullable = true)
 |-- linux: boolean (nullable = true)
 |-- rating: string (nullable = true)
 |-- positive_ratio: integer (nullable = true)
 |-- user_reviews: integer (nullable = true)
 |-- price_final: double (nullable = true)



In [ ]:
recom.printSchema()

root
 |-- app_id: integer (nullable = true)
 |-- hours: double (nullable = true)
 |-- user_id: integer (nullable = true)



In [ ]:
# Deleting unnecessary columns
dropg = ['date_release', 'discount', 'steam_deck', 'price_original']
dropr = ['helpful', 'funny', 'date', 'is_recommended', 'review_id']

games = games.drop(*dropg)
recom = recom.drop(*dropr)

In [ ]:
games.show(10)

+------+--------------------+----+-----+-----+---------------+--------------+------------+-----------+
|app_id|               title| win|  mac|linux|         rating|positive_ratio|user_reviews|price_final|
+------+--------------------+----+-----+-----+---------------+--------------+------------+-----------+
| 13500|Prince of Persia:...|true|false|false|  Very Positive|            84|        2199|       9.99|
| 22364|BRINK: Agents of ...|true|false|false|       Positive|            85|          21|       2.99|
|113020|Monaco: What's Yo...|true| true| true|  Very Positive|            92|        3722|      14.99|
|226560|  Escape Dead Island|true|false|false|          Mixed|            61|         873|      14.99|
|249050|Dungeon of the EN...|true| true|false|  Very Positive|            88|        8784|      11.99|
|250180|        METAL SLUG 3|true|false|false|  Very Positive|            90|        5579|       7.99|
|253980|             Enclave|true| true| true|Mostly Positive|           

In [ ]:
recom.show(10)

+-------+-----+-------+
| app_id|hours|user_id|
+-------+-----+-------+
| 975370| 36.3|  47199|
| 304390| 11.5|   2376|
|1085660|336.5| 230757|
| 703080| 27.4| 235736|
| 526870|  7.9|  21721|
| 306130|  8.6|  41543|
| 238960|538.8|  80787|
|    730|157.5|  57879|
| 255710| 18.7| 321815|
| 289070|397.5| 412440|
+-------+-----+-------+
only showing top 10 rows



In [ ]:
# Check null values in Data
null_games = games.select([count(when(col(i).isNull(),i)).alias(i) for i in games.columns])
null_games.show()

+------+-----+---+---+-----+------+--------------+------------+-----------+
|app_id|title|win|mac|linux|rating|positive_ratio|user_reviews|price_final|
+------+-----+---+---+-----+------+--------------+------------+-----------+
|     0|    0|  0|  0|    0|     0|             0|           0|          0|
+------+-----+---+---+-----+------+--------------+------------+-----------+



In [ ]:
null_recom = recom.select([count(when(col(i).isNull(),i)).alias(i) for i in recom.columns])
null_recom.show()

+------+-----+-------+
|app_id|hours|user_id|
+------+-----+-------+
|     0|    1|      1|
+------+-----+-------+



In [ ]:
recom = recom.dropna()
null_recom = recom.select([count(when(col(i).isNull(),i)).alias(i) for i in recom.columns])
null_recom.show()

+------+-----+-------+
|app_id|hours|user_id|
+------+-----+-------+
|     0|    0|      0|
+------+-----+-------+



In [ ]:
# Demo inputs
# Grand Theft Auto V
# Dying Light 2 Stay Human
# Cyberpunk 2077
# Red Dead Redemption 2
# Jurassic World Evolution 2
# FINAL FANTASY XIV Online
# Farming Simulator 22

In [ ]:
# Taking a input game
input_game_name = input("Enter the name of the game: ")
target_app_id = games.filter(games.title == input_game_name).collect()
target_app_id = target_app_id[0]["app_id"]
target_app_id

Enter the name of the game: Cyberpunk 2077


1091500

In [ ]:
filtered_games = games.filter(games.app_id == target_app_id)

filtered_games.show()

+-------+--------------+----+-----+-----+---------------+--------------+------------+-----------+
| app_id|         title| win|  mac|linux|         rating|positive_ratio|user_reviews|price_final|
+-------+--------------+----+-----+-----+---------------+--------------+------------+-----------+
|1091500|Cyberpunk 2077|true|false|false|Mostly Positive|            79|      541364|      59.99|
+-------+--------------+----+-----+-----+---------------+--------------+------------+-----------+



In [ ]:
# Checking how many users have played this game(app_id) from recom table
target_user_ids = recom.filter(recom.app_id == target_app_id)
target_user_ids.show(5)

+-------+-----+--------+
| app_id|hours| user_id|
+-------+-----+--------+
|1091500|108.8|  430698|
|1091500| 18.1| 3984740|
|1091500| 61.4| 6876593|
|1091500| 28.6| 8132987|
|1091500| 77.3|12235370|
+-------+-----+--------+
only showing top 5 rows



In [ ]:
# Sorting DataFrame(usr_id) by hours
# To get the players who played the game more. Because they liked the game more so they can give more accurate recommendation
# Selecting top 1000 players
usr_id = target_user_ids.orderBy("hours", ascending=False).limit(1000)
usr_id.show(5)

+-------+-----+--------+
| app_id|hours| user_id|
+-------+-----+--------+
|1091500|999.1|10611967|
|1091500|997.2| 6240929|
|1091500|996.5|12800978|
|1091500|995.7|  851016|
|1091500|994.7| 9240746|
+-------+-----+--------+
only showing top 5 rows



In [ ]:
# Extracting user_id of these users
users = usr_id.select("user_id").rdd.flatMap(lambda x: x).collect()

In [ ]:
# Checking if recom['user_id'] isin users Series
# Filter recommendations based on selected users
final_df = recom.filter(recom.user_id.isin(users))
final_df.show(5)

+-------+-----+--------+
| app_id|hours| user_id|
+-------+-----+--------+
|1091500|734.8|12653296|
| 255710|640.6| 3948193|
| 632360|100.1| 3270572|
| 976730|276.5|10334766|
|1091500|681.0|13029013|
+-------+-----+--------+
only showing top 5 rows



In [ ]:
# Counting how many users have played these games
# Checing how many users have played a game & taking the top 7 games
recom_games = final_df.groupBy("app_id").count().orderBy("count", ascending=False).limit(7).collect()
recom_games

[Row(app_id=1091500, count=1000),
 Row(app_id=377160, count=79),
 Row(app_id=275850, count=64),
 Row(app_id=1174180, count=53),
 Row(app_id=431960, count=40),
 Row(app_id=292030, count=39),
 Row(app_id=271590, count=35)]

In [ ]:
# Extract count from recom_games
count_played = [row["count"] for row in recom_games[1:]]
count_played

[79, 64, 53, 40, 39, 35]

In [ ]:
# Extract app_id from recom_games
apps = [row["app_id"] for row in recom_games[1:]]
apps

[377160, 275850, 1174180, 431960, 292030, 271590]

In [ ]:
# Making recom_games list into a DataFrame with columns as "app_id" & "count_played"
apps_count = spark.createDataFrame(recom_games, ["app_id", "count_played"])
apps_count.show()

+-------+------------+
| app_id|count_played|
+-------+------------+
|1091500|        1000|
| 377160|          79|
| 275850|          64|
|1174180|          53|
| 431960|          40|
| 292030|          39|
| 271590|          35|
+-------+------------+



In [ ]:
# Getting games name from games table by comparing app_id from apps list
output = games.filter(games.app_id.isin(apps))
output.show()

+-------+--------------------+----+-----+-----+--------------------+--------------+------------+-----------+
| app_id|               title| win|  mac|linux|              rating|positive_ratio|user_reviews|price_final|
+-------+--------------------+----+-----+-----+--------------------+--------------+------------+-----------+
| 271590|  Grand Theft Auto V|true|false|false|       Very Positive|            86|     1431104|        0.0|
| 275850|        No Man's Sky|true| true|false|     Mostly Positive|            75|      205112|      59.99|
| 292030|The Witcher® 3: W...|true|false|false|Overwhelmingly Po...|            96|      656330|      39.99|
| 377160|           Fallout 4|true|false|false|       Very Positive|            83|      198619|       6.59|
| 431960|    Wallpaper Engine|true|false|false|Overwhelmingly Po...|            98|      609312|       3.99|
|1174180|Red Dead Redempti...|true|false|false|       Very Positive|            90|      379488|      59.99|
+-------+----------

In [ ]:
# Joining two Dataframe on app_id to get count_played for each game
output = output.join(apps_count, on="app_id", how="inner")

output.show()

+-------+--------------------+----+-----+-----+--------------------+--------------+------------+-----------+------------+
| app_id|               title| win|  mac|linux|              rating|positive_ratio|user_reviews|price_final|count_played|
+-------+--------------------+----+-----+-----+--------------------+--------------+------------+-----------+------------+
| 377160|           Fallout 4|true|false|false|       Very Positive|            83|      198619|       6.59|          79|
| 275850|        No Man's Sky|true| true|false|     Mostly Positive|            75|      205112|      59.99|          64|
|1174180|Red Dead Redempti...|true|false|false|       Very Positive|            90|      379488|      59.99|          53|
| 431960|    Wallpaper Engine|true|false|false|Overwhelmingly Po...|            98|      609312|       3.99|          40|
| 292030|The Witcher® 3: W...|true|false|false|Overwhelmingly Po...|            96|      656330|      39.99|          39|
| 271590|  Grand Theft A

In [ ]:
# Filters
cond = input("Do you want to add filters(Os,Price,None): ")

if cond.lower() == 'os':
    os = input("Enter preferred Os system(windows/mac/linux/any): ").lower()
    if(os=='windows'):
      os = 'win'

    if os in ['win', 'mac', 'linux', 'any']:
        if os == 'any':
            output.show(truncate=False)
        else:
            os_col = os if os != 'any' else 'win'
            filtered_titles = output.filter(output[os_col] == True)
            if filtered_titles.count() == 0:
                print("No Games found")
            else:
                filtered_titles.show(truncate=False)
    else:
        print("Please enter a valid OS option.")

elif cond.lower() == 'price':
    price = float(input("Enter price range: "))
    filtered_titles = output.filter(output.price_final <= price)
    filtered_titles.show(truncate=False)

elif cond.lower() == 'none':
    output.show(truncate=False)

else:
    print("Please enter valid filter options.")

Do you want to add filters(Os,Price,None): price
Enter price range: 50
+------+-------------------------+----+-----+-----+-----------------------+--------------+------------+-----------+------------+
|app_id|title                    |win |mac  |linux|rating                 |positive_ratio|user_reviews|price_final|count_played|
+------+-------------------------+----+-----+-----+-----------------------+--------------+------------+-----------+------------+
|377160|Fallout 4                |true|false|false|Very Positive          |83            |198619      |6.59       |79          |
|431960|Wallpaper Engine         |true|false|false|Overwhelmingly Positive|98            |609312      |3.99       |40          |
|292030|The Witcher® 3: Wild Hunt|true|false|false|Overwhelmingly Positive|96            |656330      |39.99      |39          |
|271590|Grand Theft Auto V       |true|false|false|Very Positive          |86            |1431104     |0.0        |35          |
+------+------------------

In [ ]:
#spark.stop()